In [1]:
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
data = np.loadtxt('data_w3_ex1.csv', delimiter=',')

# split input and output to different arrays
x = data[:,0]
y = data[:,1]

# convert 1-D to 2-D array
x = np.expand_dims(x, axis=1)
y = np.expand_dims(y, axis=1)

print(f"the shape of the inputs x is: {x.shape}")
print(f"the shape of the outputs y is: {y.shape}")

the shape of the inputs x is: (50, 1)
the shape of the outputs y is: (50, 1)


In previous labs, you might have used the entire dataset to train your models. In practice however, it is best to hold out a portion of your data to measure how well your model generalizes to new examples. This will let you know if the model has overfit to your training set.

As mentioned in the lecture, it is common to split your data into three parts:

* ***training set*** - used to train the model
* ***cross validation set (also called validation, development, or dev set)*** - used to evaluate the different model configurations you are choosing from. For example, you can use this to make a decision on what polynomial features to add to your dataset.
* ***test set*** - used to give a fair estimate of your chosen model's performance against new examples. This should not be used to make decisions while you are still developing the models.

In [3]:
# 60% dataset as training data, 40% in temporary variables for now
x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.40, random_state=1)

# split 40% into dev set and test set
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

del x_, y_  # delete temp variables

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")

the shape of the training set (input) is: (30, 1)
the shape of the training set (target) is: (30, 1)
the shape of the cross validation set (input) is: (10, 1)
the shape of the cross validation set (target) is: (10, 1)
the shape of the test set (input) is: (10, 1)
the shape of the test set (target) is: (10, 1)


In [4]:
# Feature scaling

scaler_linear = StandardScaler()

# calculate mean and S.D. of the training set then transform it
X_train_scaled = scaler_linear.fit_transform(x_train)

print(f"Computed mean of the training set: {scaler_linear.mean_.squeeze():.2f}")
print(f"Computed standard deviation of the training set: {scaler_linear.scale_.squeeze():.2f}")

Computed mean of the training set: 2504.06
Computed standard deviation of the training set: 574.85


In [6]:
# training the model
linear_model = LinearRegression()

linear_model.fit(X_train_scaled, y_train)

LinearRegression()

In [7]:
# evals

# get predictions
yhat = linear_model.predict(X_train_scaled)

# MSE loss
print(f"training set MSE: {mean_squared_error(y_train, yhat) / 2}")
# we divide by 2 as sklearn's implementation only divides by m and not 2*m

training set MSE: 406.19374192533127


In [8]:
# scale dev set using mean and S.D. from train set
X_cv_scaled = scaler_linear.transform(x_cv)

yhat = linear_model.predict(X_cv_scaled)

# MSE loss
print(f"Cross validation MSE: {mean_squared_error(y_cv, yhat) / 2}")

Cross validation MSE: 551.7789026952216


In [9]:
# let's try adding polynomial features

# class to make polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)

# this will have squared values of input x
X_train_mapped = poly.fit_transform(x_train)

In [10]:
scaler_poly = StandardScaler()

# calculate mean, S.D. for squared x data
X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)

In [11]:
model = LinearRegression()

model.fit(X_train_mapped_scaled, y_train)

yhat = model.predict(X_train_mapped_scaled)
print(f"training set MSE: {mean_squared_error(y_train, yhat) / 2}")

# dev set
X_cv_mapped = poly.transform(x_cv)
X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)

yhat = model.predict(X_cv_mapped_scaled)
print(f"Cross validation MSE: {mean_squared_error(y_cv, yhat) / 2}")  # much better results

training set MSE: 49.111609334025154
Cross validation MSE: 87.69841211111911


In [12]:
# loop to do the same for models with increasing number of polynomial terms

# Initialize lists to save the errors, models, and feature transforms
train_mses = []
cv_mses = []
models = []
polys = []
scalers = []

# Loop over 10 times. Each adding one more degree of polynomial higher than the last.
for degree in range(1,11):
    
    # Add polynomial features to the training set
    poly = PolynomialFeatures(degree, include_bias=False)
    X_train_mapped = poly.fit_transform(x_train)
    polys.append(poly)
    
    # Scale the training set
    scaler_poly = StandardScaler()
    X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)
    scalers.append(scaler_poly)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train_mapped_scaled, y_train )
    models.append(model)
    
    # Compute the training MSE
    yhat = model.predict(X_train_mapped_scaled)
    train_mse = mean_squared_error(y_train, yhat) / 2
    train_mses.append(train_mse)
    
    # Add polynomial features and scale the cross validation set
    X_cv_mapped = poly.transform(x_cv)
    X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)
    
    # Compute the cross validation MSE
    yhat = model.predict(X_cv_mapped_scaled)
    cv_mse = mean_squared_error(y_cv, yhat) / 2
    cv_mses.append(cv_mse)

In [13]:
# selecting the best model, with lowest dev and train set loss
degree = np.argmin(cv_mses) + 1  # add 1 because indices start at 0
print(f"Lowest CV MSE is found in the model with degree={degree}")

Lowest CV MSE is found in the model with degree=4


In [14]:
# using model with degree 4

# Add polynomial features to the test set
X_test_mapped = polys[degree-1].transform(x_test)

# Scale the test set
X_test_mapped_scaled = scalers[degree-1].transform(X_test_mapped)

# Compute the test MSE
yhat = models[degree-1].predict(X_test_mapped_scaled)
test_mse = mean_squared_error(y_test, yhat) / 2

print(f"Training MSE: {train_mses[degree-1]:.2f}")
print(f"Cross Validation MSE: {cv_mses[degree-1]:.2f}")
print(f"Test MSE: {test_mse:.2f}")

Training MSE: 47.15
Cross Validation MSE: 79.43
Test MSE: 104.63
